In [1]:
import math
import keras_ocr
import matplotlib.pyplot as plt
import tensorflow as tf

In [21]:
recognizer_alphabet = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"
recogniser = keras_ocr.recognition.Recognizer()
# recogniser.model.load_weights(f"C:\\Users\\M-S-I\\Documents\\IBDA Semester 8\\Skripsi\\Machine Learning\\model\\keras_ocr_weights\\recognizer_cocotext_mid_freeze.h5")

# Initialize pipeline
pipeline = keras_ocr.pipeline.Pipeline(recognizer=recogniser)

# Read in image
# image_paths = [
#     "C:\\Users\\M-S-I\\Documents\\IBDA Semester 8\\Skripsi\\Web\\pdf2png\\Proposal Proyek Akhir - Google Docs\\page_2.png",
#     "C:\\Users\\M-S-I\\Documents\\IBDA Semester 8\\Skripsi\\Web\\pdf2png\\Proposal Proyek Akhir - Google Docs\\page_2.png"
# ]
# images = [keras_ocr.tools.read(img) for img in image_paths]
images = keras_ocr.tools.read("C:\\Users\\M-S-I\\Documents\\IBDA Semester 8\\OCR Image\\tes char.png")

# prediction_groups is a list of (word, box) tuples
prediction_groups = pipeline.recognize([images])
# prediction_groups[0]

Looking for C:\Users\M-S-I\.keras-ocr\crnn_kurapan.h5
Looking for C:\Users\M-S-I\.keras-ocr\craft_mlt_25k.h5
1/1 [==============================] - 2s 2s/step


In [ ]:
fig, axs = plt.subplots(nrows=len(images), figsize=(20, 20))
for ax, image, predictions in zip(axs, images, prediction_groups):
    keras_ocr.tools.drawAnnotations(image=image, predictions=predictions, ax=ax)

In [22]:
raw_detections = []
for prediction in prediction_groups[0]:
    raw_detections.append(prediction[0])
print(f'Detections: {raw_detections}')

Detections: ['dan', 'dunia', 'berputar', 'ditengah', 'semesta', 'alam', 'dan', 'luas', 'gelap', 'yang']


In [23]:
def get_distance(predictions):
    """
    Function returns dictionary with (key,value):
        * text : detected text in image
        * center_x : center of bounding box (x)
        * center_y : center of bounding box (y)
        * distance_from_origin : hypotenuse
        * distance_y : distance between y and origin (0,0)
    """

    # Point of origin
    x0, y0 = 0, 0

    # Generate dictionary
    detections = []
    for group in predictions:

        # Get center point of bounding box
        top_left_x, top_left_y = group[1][0]
        bottom_right_x, bottom_right_y = group[1][1]
        center_x, center_y = (top_left_x + bottom_right_x)/2, (top_left_y + bottom_right_y)/2

        # Use the Pythagorean Theorem to solve for distance from origin
        distance_from_origin = math.dist([x0,y0], [center_x, center_y])

        # Calculate difference between y and origin to get unique rows
        distance_y = center_y - y0

        # Append all results
        detections.append({
                            'text': group[0],
                            'center_x': center_x,
                            'center_y': center_y,
                            'distance_from_origin': distance_from_origin,
                            'distance_y': distance_y
                        })

    return detections

In [24]:
predictions = prediction_groups[0] # extract text list
predictions = get_distance(predictions)
# predictions

In [25]:
def distinguish_rows(lst, thresh=15):
    """Function to help distinguish unique rows"""
    sublists = []
    for i in range(0, len(lst)-1):
        if (lst[i+1]['distance_y'] - lst[i]['distance_y'] <= thresh):
            if lst[i] not in sublists:
                sublists.append(lst[i])
            sublists.append(lst[i+1])
        else:
            yield sublists
            sublists = [lst[i+1]]
    yield sublists

In [26]:
# Set thresh higher for text further apart
predictions = list(distinguish_rows(predictions, thresh=15))
predictions

[[{'text': 'dan',
   'center_x': 25.0,
   'center_y': 4.0,
   'distance_from_origin': 25.317977802344327,
   'distance_y': 4.0},
  {'text': 'dunia',
   'center_x': 93.5,
   'center_y': 5.0,
   'distance_from_origin': 93.63359439859178,
   'distance_y': 5.0},
  {'text': 'berputar',
   'center_x': 191.6936798095703,
   'center_y': 5.60459041595459,
   'distance_from_origin': 191.77559363137084,
   'distance_y': 5.60459041595459},
  {'text': 'ditengah',
   'center_x': 305.0,
   'center_y': 5.0,
   'distance_from_origin': 305.0409808533929,
   'distance_y': 5.0},
  {'text': 'semesta',
   'center_x': 417.4999694824219,
   'center_y': 6.0,
   'distance_from_origin': 417.54308103215317,
   'distance_y': 6.0}],
 [{'text': 'alam',
   'center_x': 29.5,
   'center_y': 42.0,
   'distance_from_origin': 51.324945202113945,
   'distance_y': 42.0},
  {'text': 'dan',
   'center_x': 195.3096466064453,
   'center_y': 42.66497802734375,
   'distance_from_origin': 199.9153781168629,
   'distance_y': 42.664

In [27]:
# Remove all empty rows
predictions = list(filter(lambda x:x!=[], predictions))

In [28]:
# Order text detections in human readable format
ordered_preds = []
for row in predictions:
    row = sorted(row, key=lambda x:x['distance_from_origin'])
    for each in row: ordered_preds.append(each['text'])
print(f'Detections: {ordered_preds}')

Detections: ['dan', 'dunia', 'berputar', 'ditengah', 'semesta', 'alam', 'yang', 'luas', 'dan', 'gelap']


In [29]:
# Join detections into sentence
chunk_size = 16
result = "\n".join([" ".join(ordered_preds[i:i+chunk_size]) for i in range(0, len(ordered_preds), chunk_size)])
print(result)

dan dunia berputar ditengah semesta alam yang luas dan gelap
